In [1]:
import glob
import pandas as pd

rawData = []
outfile = "results.pkl"

In [2]:
for file in glob.glob("*/*.log"):
    print(file)
    workloadParts = (file.split("/")[1]).split("_")
    platform = workloadParts[0]
    functionType = (workloadParts[1])
    regression = int(workloadParts[2])
    #    with open(file) as logfile:
    with open(file) as logfile:
        for line in logfile:
            # for DuetFaaS
            if line.__contains__("isColdStart: true"):
                print("skipping due to cold start")
                continue
            if line.__contains__("Response Time"):
                
                f1Durations = []
                f2Durations = []
                line = line.strip()
                parts = line.split(" ")
                if "." in parts[4]:
                    f1 = int(parts[4].split(".")[0])
                else:
                    f1 = int(parts[4][0:parts[4].index("m")])
                if "." in parts[18]:
                    f2 = int(parts[18].split(".")[0])
                else:
                    f2 = int(parts[18][0:parts[18].index("m")])
                #print(f" f1 is {f1}, f2 is {f2}")
                f1Durations.append(f1)
                f2Durations.append(f2)
                row = {}
                row["platform"] = "aws"
                row["functionType"] = functionType
                row["regression"] = regression
                row["mode"] = "duet"
                #print(f"platform is {platform}, functionType is {functionType}, regression is {regression}")
                for i in range(0, len(f1Durations)):
                    row["f1"] = f1Durations[i]
                    row["f2"] = f2Durations[i]
                    row["change"] = f2Durations[i] / f1Durations[i]
                    rawData.append(row)
            # for faasterBench
            if line.__contains__("faaster"):
                if(file.__contains__("aws")):
                    line = line[:-4]
                    if line.__contains__("modeA") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeA"

                        f1Durations = []
                        f2Durations = []
                        f2Found = False

                        for i in range(4,len(parts2)):
                            if parts2[i] == "f2":
                                break
                            
                            #if int(parts2[i]) < 2000 and int(parts2[i+4]) < 2100:
                            f1Durations.append(int(parts2[i]))
                            f2Durations.append(int(parts2[i+4]))
                        for i in range(0, len(f1Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = f1Durations[i]
                            row["f2"] = f2Durations[i]
                            row["change"] = f2Durations[i] / f1Durations[i]
                            rawData.append(row)
                    if line.__contains__("modeB") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeB"
                        f1Durations = []

                        for i in range(4,len(parts2)):
                            if parts2[i] == "f2":
                                break
                            else:
                                if int(parts2[i]) < 2000:
                                    f1Durations.append(int(parts2[i]))

                        for i in range(0, len(f1Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = f1Durations[i]
                            row["f2"] = 0
                            row["change"] = 0
                            rawData.append(row)

                    if line.__contains__("modeC") and not (line.__contains__("cold start")):
                        line = line.strip()
                        parts2 = line.split(" ")
                        #print(line)
                        row = {}
                        row["platform"] = platform
                        row["functionType"] = functionType
                        row["regression"] = regression
                        row["mode"] = "modeC"
                        f2Durations = []

                        for i in range(5,len(parts2)):
                            if parts2[i] == "\",":
                                break
                            else:
                                if int(parts2[i]) < 2100:
                                    f2Durations.append(int(parts2[i]))

                        for i in range(0, len(f2Durations)):
                            #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                            row["f1"] = 0
                            row["f2"] = f2Durations[i]
                            row["change"] = 0
                            rawData.append(row)
                if(file.__contains__("google")):
                    if line.__contains__("textPayload"):
                        if line.__contains__("modeA") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeA"

                            f1Durations = []
                            f2Durations = []
                            f2Found = False

                            for i in range(4,len(parts2)):
                                if parts2[i] == "f2":
                                    break
                                
                                else:
                                    if int(parts2[i]) < 1800 and int(parts2[i+4]) < 1800:
                                        f1Durations.append(int(parts2[i]))
                                        f2Durations.append(int(parts2[i+4]))

                            for i in range(0, len(f1Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = f1Durations[i]
                                row["f2"] = f2Durations[i]
                                row["change"] = f2Durations[i] / f1Durations[i]
                                rawData.append(row)
                        if line.__contains__("modeB") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeB"

                            f1Durations = []

                            for i in range(4,len(parts2)):
                                if parts2[i] == "f2":
                                    break
                                else:
                                    if int(parts2[i]) < 1800:
                                        f1Durations.append(int(parts2[i]))

                            for i in range(0, len(f1Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = f1Durations[i]
                                row["f2"] = 0
                                row["change"] = 0
                                rawData.append(row)

                        if line.__contains__("modeC") and not line.__contains__("cold start"):
                            line = line.strip()
                            parts2 = line.split(" ")
                            #print(line)
                            row = {}
                            row["platform"] = platform
                            row["functionType"] = functionType
                            row["regression"] = regression
                            row["mode"] = "modeC"

                            f2Durations = []

                            for i in range(5,len(parts2)):
                                if parts2[i] == "\",":
                                    break
                                else:
                                    if int(parts2[i]) < 1800:
                                        f2Durations.append(int(parts2[i]))

                            for i in range(0, len(f2Durations)):
                                #if (f1Durations[i] > 200  and f2Durations[i] > 200):
                                row["f1"] = 0
                                row["f2"] = f2Durations[i]
                                row["change"] = 0
                                rawData.append(row)



duet/duet_mem_0_5_11_18.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
duet/duet_mem_5_5_11_18.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
duet/duet_cpu_5_5_11_18.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
duet/duet_cpu_0_5_11_18.log
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
skipping due to cold start
aws/aws_cpu_0_05_02_15.log
aws/aws_mem_5_05_02_15.log
aws/aws_mem_0_05_02_15.log
aws/aws_cpu_5_05_02_15.log


In [3]:
df_results = pd.DataFrame(rawData)
df_results.head()
df_results.describe()
print(df_results.loc[(df_results["mode"] == "duet")&
                     (df_results["functionType"] == "mem")&
                     (df_results["regression"] == 5)])
df_results.to_pickle(outfile)
#df_results.to_csv("nils2.csv")

     platform functionType  regression  mode  f1  f2    change
1493      aws          mem           5  duet  45  46  1.022222
1494      aws          mem           5  duet  42  45  1.071429
1495      aws          mem           5  duet  63  66  1.047619
1496      aws          mem           5  duet  60  64  1.066667
1497      aws          mem           5  duet  63  64  1.015873
...       ...          ...         ...   ...  ..  ..       ...
2981      aws          mem           5  duet  44  45  1.022727
2982      aws          mem           5  duet  43  36  0.837209
2983      aws          mem           5  duet  42  45  1.071429
2984      aws          mem           5  duet  42  46  1.095238
2985      aws          mem           5  duet  43  45  1.046512

[1493 rows x 7 columns]
